In [306]:
import requests
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
import os.path
import glob
import time
import pandas as pd
from datetime import datetime

In [55]:
# def get_html_code(job_name, page_num):
    save_path = r'C:\Users\micha\project\data\raw'
    completeName = os.path.join(save_path, f"{job_name}_{page_num}.html")
    
#     browser = Chrome(r'C:\Users\micha\project\drivers\chromedriver')
#     browser.get(f'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"{job_name}"&page={page_num}')
#     t = browser.page_source
    
    with open(completeName, 'w', encoding="utf-8") as my_file:
        my_file.write(t)

In [146]:
def get_html_code(job_name, page_num):
    browser = Chrome(r'C:\Users\micha\project\drivers\chromedriver')
    browser.get("view-source:" + f'https://nofluffjobs.com/pl/jobs/?criteria=keyword%3D"{job_name}"&page={page_num}')
    html = browser.find_element_by_tag_name('body').text
    
    return html

In [90]:
def any_offers_left(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
        
    main_container = soup.select_one('nfj-main-content')
    postings_container = main_container.select_one('nfj-postings-list')
    list_container = postings_container.select_one('div.list-container')
    
    if list_container == None:
        return False
    else:
        return True

In [147]:
def save_html_codes(job_name):
    i = 1
    html = get_html_code(job_name, i)
    
    while True:
        time.sleep(3)
        
        html = get_html_code(job_name, i)
        
        if not any_offers_left(html):
            break
        
        save_path = r'C:\Users\micha\project\data\raw'
        completeName = os.path.join(save_path, f"{job_name}_{i}.html")
        
        with open(completeName, 'w', encoding="utf-8") as my_file:
            my_file.write(html)
        i+=1

In [150]:
save_html_codes('data engineer')

<ipython-input-146-32131a440a08>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = Chrome(r'C:\Users\micha\project\drivers\chromedriver')
<ipython-input-146-32131a440a08>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  html = browser.find_element_by_tag_name('body').text


In [214]:
def get_offer_info(html):
    html_offers_list = []
    
    with open(html, encoding='utf-8') as html:
        soup = BeautifulSoup(html, 'html.parser')
    
    main_container = soup.select_one('nfj-main-content')
    postings_container = main_container.select_one('nfj-postings-list')
    list_container = postings_container.select_one('div.list-container')
    items_container = list_container.select('a.posting-list-item')
    job = soup.select_one('nfj-search-box')
    
    for elem in range(len(items_container)):
        offer = []
        offer.append(items_container[elem]) # kod html zawierajacy info o ofercie
        offer.append(job)                   # kod html zawierajacy info o hasle wyszukiwania
        html_offers_list.append(offer)
    
    return html_offers_list

In [339]:
def offer_data_dict(offer_code):
    
    name = offer_code[0].select_one('h3.posting-title__position').text.strip()
    company = offer_code[0].select_one('span.d-block').text.strip().replace('@ ', '')
    technology = offer_code[0].select_one('nfj-posting-item-tags').text.split('PLN')[1].strip()
    job = offer_code[1].select_one('mat-chip.mat-chip').text.strip()
    localization = offer_code[0].select_one('nfj-posting-item-city')
    if localization == None:
        city = 'Zdalna'
        country = 'N/A'
    else:
        localization = localization.text.strip()
        city = localization.split(', ')[0].strip()
        country = localization.split(', ')[1].strip()
    
    salary = offer_code[0].select_one('span.text-truncate.badgy.salary')
    salary_formatted = salary.text.strip().split('-')
    low = salary_formatted[0].strip()
    high = salary_formatted[1][:-3].strip()
    currency = salary_formatted[1][-3:].strip()
    
    return {
        'name': name,
        'company': company,
        'technology': technology,
        'job': job,
        'location': {'city': city, 'country': country},
        'salary': {'low': low, 'high': high, 'currency': currency} 
        }

In [340]:
files_list = glob.glob(r'C:\Users\micha\project\data\raw\**.html')
offers_in_html = []
offers_dict_list = []
for file in files_list:
    offers_in_html.append(get_offer_info(file))

for elem in range(len(offers_in_html)):
    for element in range(len(offers_in_html[elem])):
        offers_dict_list.append(offer_data_dict(offers_in_html[elem][element]))

In [347]:
df = pd.json_normalize(offers_dict_list, sep='_')
df

,name,company,technology,job,location_city,location_country,salary_low,salary_high,salary_currency
0,Data Scientist,Kontomatik Sp. z o.o.,python,data analyst,Warszawa,POL,8 000,12 000,PLN
1,Data Analyst,Ringier Axel Springer Tech,,data analyst,Zdalna,N/A,9 000,13 000,PLN
2,Senior Data Mining Analyst,McKinsey & Company,,data analyst,Wrocław,POL,9 000,14 000,PLN
3,Data Analyst,Devire,,data analyst,Zdalna,N/A,8 000,16 000,PLN
4,Data Analyst,Globitell Wholesale Sp. z o.o. Sp. K.,,data analyst,Warszawa,POL,6 000,9 000,PLN
...,...,...,...,...,...,...,...,...,...
88,Data Scientist,Syncron Sp. z o.o.,python,data scientist,Zdalna,N/A,15 700,18 800,PLN
89,(Senior) Data Scientist,Pearson,Python,data scientist,Poznań,POL\n + 1,12 500,25 000,PLN
90,Mid Data Scientist,LAIT,python,data scientist,Warsaw,POL,15 000,18 000,PLN
91,Data Scientist,Onwelo S.A.,,data scientist,Zdalna,N/A,13 650,17 850,PLN


In [352]:
df.to_csv(
         fr'C:\Users\micha\project\data\interim\job_offers_{datetime.today().strftime("%d-%m-%Y")}.csv',
        sep=';',
        encoding='utf-8',
        index=False
)



In [354]:
df_1 = pd.read_csv(
    r'C:\Users\micha\project\data\interim\job_offers_14-11-2021.csv',
    sep=';',
    encoding='utf-8',
)
df_1

,name,company,technology,job,location_city,location_country,salary_low,salary_high,salary_currency
0,Data Scientist,Kontomatik Sp. z o.o.,python,data analyst,Warszawa,POL,8 000,12 000,PLN
1,Data Analyst,Ringier Axel Springer Tech,NaN,data analyst,Zdalna,NaN,9 000,13 000,PLN
2,Senior Data Mining Analyst,McKinsey & Company,NaN,data analyst,Wrocław,POL,9 000,14 000,PLN
3,Data Analyst,Devire,NaN,data analyst,Zdalna,NaN,8 000,16 000,PLN
4,Data Analyst,Globitell Wholesale Sp. z o.o. Sp. K.,NaN,data analyst,Warszawa,POL,6 000,9 000,PLN
...,...,...,...,...,...,...,...,...,...
88,Data Scientist,Syncron Sp. z o.o.,python,data scientist,Zdalna,NaN,15 700,18 800,PLN
89,(Senior) Data Scientist,Pearson,Python,data scientist,Poznań,POL\n + 1,12 500,25 000,PLN
90,Mid Data Scientist,LAIT,python,data scientist,Warsaw,POL,15 000,18 000,PLN
91,Data Scientist,Onwelo S.A.,NaN,data scientist,Zdalna,NaN,13 650,17 850,PLN
